A place to test random stuff

In [11]:
from monkey.model.classification_model.efficientnet_b0 import (
    EfficientNet_B0,
)
import torch
import torch.nn as nn
from torchsummary import summary
from monkey.model.utils import (
    get_classification_metrics,
    get_activation_function,
)
from monkey.model.loss_functions import get_loss_function, dice_coeff
import numpy as np
from pprint import pprint

In [ ]:
model = EfficientNet_B0(input_channels=3, num_classes=1)
model = model.eval()
summary(model, input_size=(3, 32, 32))
print(model)

In [ ]:
test = torch.randn(size=(4, 3, 32, 32), dtype=torch.float32)
gt = torch.tensor([0, 0, 0, 1], dtype=torch.float)
gt = torch.unsqueeze(gt, 1)

dice_loss_fn = nn.BCELoss()

pred_labels = []

act = get_activation_function("sigmoid")

with torch.no_grad():
    out = model(test)
    print(out.shape)
    out = act(out)
    # out = torch.squeeze(out)
    pred = (out > 0.5).float()
    pred = torch.squeeze(pred)
    print(pred)
    pred = pred.cpu().tolist()

    pred_labels.extend(pred)

    dice_loss = dice_loss_fn(out, gt)


gt_labels = torch.squeeze(gt).cpu().tolist()

print(f"probs {out}")
print(out.shape)
print(f"gt {gt}")
print(gt.shape)
print(f"loss {dice_loss}")
# print(f"weighted loss {weighted_loss}")

print(f"pred labels: {pred_labels}")
print(f"true labels: {gt_labels}")

metrics = get_classification_metrics(gt_labels, pred_labels)
print(metrics)

In [ ]:
true_mask = np.zeros(shape=(1, 2, 256, 256), dtype=np.float32)
true_mask[0, 0, :, :] = 1
true_mask = torch.tensor(
    true_mask, dtype=torch.float32, requires_grad=True
)

# pprint(true_mask)
pred_mask = np.zeros(shape=(1, 2, 256, 256), dtype=np.float32)
pred_mask[0, 0, :, :] = 0.9
pred_mask[0, 1, :, 0:128] = 0
pred_mask = torch.tensor(
    pred_mask, dtype=torch.float32, requires_grad=True
)

dice_loss_fn = get_loss_function("Dice")
dice_loss_fn.set_multiclass(True)

dice_loss = dice_loss_fn.compute_loss(true_mask, pred_mask)

pprint(dice_loss)

In [ ]:
mat_a = torch.zeros(size=(1, 1, 4, 4))
mat_a[0, 0, 0:2, 0:2] = 1
mat_b = torch.zeros(size=(1, 1, 4, 4))
mat_b[0, 0, 0:2, 0:2] = 0.5
epsilon = 1e-11
non_zero_overlap = torch.dot(mat_a.reshape(-1), mat_b.reshape(-1))
pprint(non_zero_overlap)
sets_sum = torch.sum(mat_a) + torch.sum(mat_b)
pprint(sets_sum)
if non_zero_overlap == 0:
    sets_sum = 1
sim = (2 * non_zero_overlap + epsilon) / (sets_sum + epsilon)

dice = dice_coeff(mat_a, mat_b, reduce_batch_first=True)
pprint(sim)

In [14]:
from torchvision.transforms.functional import resize

patch = np.zeros(shape=(32, 32, 3))
patch = np.moveaxis(patch, 2, 0)
patch = torch.tensor(patch, dtype=torch.float)
patch = resize(patch, (224, 224))
print(patch.shape)
patch = torch.unsqueeze(patch, dim=0)


print(patch.shape)

torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])


/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
